# CA Coronavirus Cases and Deaths Trends

CA's [Blueprint for a Safer Economy](https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyMonitoringOverview.aspx) assigns each county [to a tier](https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyMonitoringOverview.aspx) based on case rate and test positivity rate. What's opened / closed [under each tier](https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH%20Document%20Library/COVID-19/Dimmer-Framework-September_2020.pdf).

Tiers, from most severe to least severe, categorizes coronavirus spread as <strong><span style='color:#6B1F84'>widespread; </span></strong>
<strong><span style='color:#F3324C'>substantial; </span></strong><strong><span style='color:#F7AE1D'>moderate; </span></strong><strong><span style = 'color:#D0E700'>or minimal.</span></strong>
**Counties must stay in the current tier for 3 consecutive weeks and metrics from the last 2 consecutive weeks must fall into less restrictive tier before moving into a less restrictive tier.**

We show *only* case charts labeled with each county's population-adjusted tier cut-offs.

**Related daily reports:** 
1. **[US counties report on cases and deaths for select major cities](https://cityoflosangeles.github.io/covid19-indicators/us-county-trends.html)**
1. **[Los Angeles County, detailed indicators](https://cityoflosangeles.github.io/covid19-indicators/coronavirus-stats.html)**
1. **[Los Angeles County neighborhoods report on cases and deaths](https://cityoflosangeles.github.io/covid19-indicators/la-neighborhoods-trends.html)**

Code available in GitHub: [https://github.com/CityOfLosAngeles/covid19-indicators](https://github.com/CityOfLosAngeles/covid19-indicators)
<br>
Get informed with [public health research](https://github.com/CityOfLosAngeles/covid19-indicators/blob/master/reopening-sources.md)

In [ ]:
import altair as alt
import altair_saver
import geopandas as gpd
import os
import pandas as pd

import default_parameters
import make_charts
import make_maps
import neighborhood_utils
import us_county_utils
import utils

from datetime import date, datetime, timedelta
from IPython.display import display_html, Markdown, HTML, Image

# For map
import branca.colormap
import ipywidgets
# There's a warning that comes up about projects, suppress
import warnings
warnings.filterwarnings("ignore")

# Default parameters
time_zone = default_parameters.time_zone
start_date = datetime(2021, 3, 1).date()
today_date = default_parameters.today_date

fulldate_format = default_parameters.fulldate_format

#alt.renderers.enable('html')

In [ ]:
STATE = "CA"

jhu = us_county_utils.clean_jhu(start_date)
jhu = jhu[jhu.state_abbrev==STATE]

hospitalizations = us_county_utils.clean_hospitalizations(start_date)

vaccinations = utils.clean_vaccines_by_county()
vaccinations_demog = utils.clean_vaccines_by_demographics()

ca_counties = list(jhu[jhu.state_abbrev==STATE].county.unique()) 

# Put LA county first
ca_counties.remove("Los Angeles")
ca_counties = ["Los Angeles"] + ca_counties

data_through = jhu.date.max()

In [ ]:
display(Markdown(
        f"Report updated: {default_parameters.today_date.strftime(fulldate_format)}; "
        f"data available through {data_through.strftime(fulldate_format)}."
    )
)

In [ ]:
title_font_size = 9

def plot_charts(cases_df, hospital_df, vaccine_df, vaccine_demog_df, county_name):
    cases_df = cases_df[cases_df.county==county_name]
    hospital_df = hospital_df[hospital_df.county==county_name]
    vaccine_df = vaccine_df[vaccine_df.county==county_name]
    vaccine_df2 = vaccine_demog_df[vaccine_demog_df.county==county_name]
    
    name = cases_df.county.iloc[0]
    
    cases_chart, deaths_chart = make_charts.setup_cases_deaths_chart(cases_df, "county", name)
    hospitalizations_chart = make_charts.setup_county_covid_hospital_chart(
        hospital_df.drop(columns = "date"), county_name)
    
    vaccines_type_chart = make_charts.setup_county_vaccination_doses_chart(vaccine_df, county_name)
    vaccines_pop_chart = make_charts.setup_county_vaccinated_population_chart(vaccine_df, county_name)
    vaccines_age_chart = make_charts.setup_county_vaccinated_category(vaccine_df2, county_name, category="Age Group")
    
    outbreak_chart = (alt.hconcat(
                        cases_chart, 
                        deaths_chart, 
                        make_charts.add_tooltip(hospitalizations_chart, "hospitalizations")
                    ).configure_concat(spacing=50)
                     )
    
    #https://stackoverflow.com/questions/60328943/how-to-display-two-different-legends-in-hconcat-chart-using-altair
    vaccines_chart = (alt.hconcat(
                        make_charts.add_tooltip(vaccines_type_chart, "vaccines_type"), 
                        make_charts.add_tooltip(vaccines_pop_chart, "vaccines_pop"), 
                        make_charts.add_tooltip(vaccines_age_chart, "vaccines_age"),
                    ).resolve_scale(color="independent")
                      .configure_view(stroke=None)
                      .configure_concat(spacing=0)
                     )
        
    outbreak_chart = (make_charts.configure_chart(outbreak_chart)
              .configure_title(fontSize=title_font_size)
            )
    
    vaccines_chart = (make_charts.configure_chart(vaccines_chart)
                      .configure_title(fontSize=title_font_size)
                     )
    
    county_state_name = county_name + f", {STATE}"
    display(Markdown(f"#### {county_state_name}"))
    try:
        us_county_utils.county_caption(cases_df, county_name)
    except:
        pass
    us_county_utils.ca_hospitalizations_caption(hospital_df, county_name)
    us_county_utils.ca_vaccinations_caption(vaccine_df, county_name)
    
    make_charts.show_svg(outbreak_chart)
    make_charts.show_svg(vaccines_chart)


In [ ]:
display(Markdown("<strong>Cases chart, explained</strong>"))
Image("../notebooks/chart_parts_explained.png", width=700)

<a id='counties_by_region'></a>

## Counties by Region
<strong>Superior California Region: </strong> [Butte](#Butte), Colusa, 
[El Dorado](#El-Dorado), 
Glenn, 
[Lassen](#Lassen), Modoc, 
[Nevada](#Nevada), 
[Placer](#Placer), Plumas, 
[Sacramento](#Sacramento), 
[Shasta](#Shasta), Sierra, Siskiyou, 
[Sutter](#Sutter), 
[Tehama](#Tehama), 
[Yolo](#Yolo), 
[Yuba](#Yuba)
<br>
<strong>North Coast:</strong> [Del Norte](#Del-Norte), 
[Humboldt](#Humboldt), 
[Lake](#Lake), 
[Mendocino](#Mendocino), 
[Napa](#Napa), 
[Sonoma](#Sonoma), Trinity
<br>
<strong>San Francisco Bay Area:</strong> [Alameda](#Alameda), 
[Contra Costa](#Contra-Costa), 
[Marin](#Marin), 
[San Francisco](#San-Francisco), 
[San Mateo](#San-Mateo), 
[Santa Clara](#Santa-Clara), 
[Solano](#Solano)
<br>
<strong>Northern San Joaquin Valley:</strong> Alpine, Amador, Calaveras, 
[Madera](#Madera), Mariposa, 
[Merced](#Merced), 
Mono, 
[San Joaquin](#San-Joaquin), 
[Stanislaus](#Stanislaus), 
[Tuolumne](#Tuolumne)
<br>
<strong>Central Coast:</strong> [Monterey](#Monterey), 
[San Benito](#San-Benito), 
[San Luis Obispo](#San-Luis-Obispo), 
[Santa Barbara](#Santa-Barbara), 
[Santa Cruz](#Santa-Cruz), 
[Ventura](#Ventura)
<br>
<strong>Southern San Joaquin Valley:</strong> [Fresno](#Fresno), 
Inyo, 
[Kern](#Kern), 
[Kings](#Kings), 
[Tulare](#Tulare)
<br>
<strong>Southern California:</strong> [Los Angeles](#Los-Angeles), 
[Orange](#Orange), 
[Riverside](#Riverside), 
[San Bernardino](#San-Bernardino)
<br>
<strong>San Diego-Imperial:</strong> [Imperial](#Imperial), 
[San Diego](#San-Diego)
<br>
<br>
[**Summary of CA County Severity Map**](#summary)
<br>
[**Vaccinations by Zip Code**](#vax_map)

Note for <i>small values</i>: If the 7-day rolling average of new cases or new deaths is under 10, the 7-day rolling average is listed for the past week, rather than a percent change. Given that it is a rolling average, decimals are possible, and are rounded to 1 decimal place. Similarly for hospitalizations.

In [ ]:
for c in ca_counties:
    id_anchor = c.replace(" - ", "-").replace(" ", "-")

    display(HTML(f"<a id={id_anchor}></a>"))
    plot_charts(jhu, hospitalizations, vaccinations, vaccinations_demog, c)
    display(HTML(
        "<br>"
        "<a href=#counties_by_region>Return to top</a><br>"
    ))

<a id=summary></a>

## Summary of CA Counties

In [ ]:
ca_boundary = gpd.read_file("s3://public-health-dashboard/jhu_covid19/ca_counties_boundary.geojson")

def grab_map_stats(df):
    # Let's grab the last available date for each county
    df = (df.sort_values(["county", "fips", "date2"], 
                         ascending = [True, True, False])
          .drop_duplicates(subset = ["county", "fips"], keep = "first")
          .reset_index(drop=True)
         )
    
    # Calculate its severity metric
    df = df.assign(
        severity = (df.cases_avg7 / df.tier1_case_cutoff).round(1)
    )
    
    # Make gdf
    gdf = pd.merge(ca_boundary, df, 
                   on = ["fips", "county"], how = "left", validate = "1:1")
    gdf = gdf.assign(
        cases_avg7 = gdf.cases_avg7.round(1),
        deaths_avg7 = gdf.deaths_avg7.round(1),
    )
    return gdf

gdf = grab_map_stats(jhu)

#### Severity by County
Severity measured as proportion relative to Tier 1 (minimal) threshold.
<br>*1 = at Tier 1 threshold*
<br>*2 = 2x higher than Tier 1 threshold*

In [ ]:
MAX_SEVERITY = gdf.severity.max()

light_gray = make_charts.light_gray

#https://stackoverflow.com/questions/47846744/create-an-asymmetric-colormap
"""
Against Tier 4 cut-off
If severity = 1 when case_rate = 7 per 100k
If severity = x when case_rate = 4 per 100k
If severity = y when case_rate = 1 per 100k
x = 4/7; y = 1/7

colormap_cutoff = [
    (1/7), (4/7), 1, 2.5, 5
]

Against Tier 1 cut-off
If severity = 1 when case_rate = 1 per 100k
If severity = x when case_rate = 4 per 100k
If severity = y when case_rate = 7 per 100k
x = 4; y = 7
""" 

# Note: CA reopening guidelines have diff thresholds based on how many vaccines are administered...
# We don't have vaccine info, so ignore, use original cut-offs
colormap_cutoff = [
    1, 4, 7, 10, 15
]

colorscale = branca.colormap.StepColormap(
                colors=["#D0E700", "#F7AE1D", "#F77889", 
                        "#D59CE8", "#B249D4", "#6B1F84", # purples
                       ], 
                index=colormap_cutoff,
                vmin=0, vmax=MAX_SEVERITY,
)

popup_dict = {
    "county": "County",
    "severity": "Severity", 
}

tooltip_dict = {
    "county": "County: ",
    "severity": "Severity: ",
    "new_cases": "New Cases Yesterday: ", 
    "cases_avg7": "New Cases (7-day rolling avg): ",
    "new_deaths": "New Deaths Yesterday: ", 
    "deaths_avg7": "New Deaths (7-day rolling avg): ",
    "cases": "Cumulative Cases", 
    "deaths": "Cumulative Deaths", 
}


fig = make_maps.make_choropleth_map(gdf.drop(columns = ["date", "date2"]), 
                                    plot_col = "severity", 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = tooltip_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 570, fig_height = 700, 
                                    zoom=6, centroid = [36.2, -119.1])

In [ ]:
display(Markdown("Severity Scale"))
display(colorscale)
fig

In [ ]:
table = (gdf[gdf.severity.notna()]
         [["county", "severity"]]
         .sort_values("severity", ascending = False)
         .reset_index(drop=True)
        )

df1_styler = (table.iloc[:14].style.format({'severity': "{:.1f}"})
              .set_table_attributes("style='display:inline'")
              #.set_caption('Caption table 1')
              .hide_index()
             )
df2_styler = (table.iloc[15:29].style.format({'severity': "{:.1f}"})
              .set_table_attributes("style='display:inline'")
              #.set_caption('Caption table 2')
              .hide_index()
             )
df3_styler = (table.iloc[30:].style.format({'severity': "{:.1f}"})
              .set_table_attributes("style='display:inline'")
              #.set_caption('Caption table 2')
              .hide_index()
             )

display(Markdown("#### Counties (in order of decreasing severity)"))
display_html(df1_styler._repr_html_() + 
             df2_styler._repr_html_() + 
             df3_styler._repr_html_(), raw=True)
             

[Return to top](#counties_by_region)

In [ ]:
# Vaccination data by zip code
def select_latest_date(df):
    df = (df[df.date == df.date.max()]
          .sort_values(["county", "zipcode"])
          .reset_index(drop=True)
         )
    
    return df

vax_by_zipcode = neighborhood_utils.clean_zipcode_vax_data()
vax_by_zipcode = select_latest_date(vax_by_zipcode)

In [ ]:
popup_dict = {
    "county": "County",
    "zipcode": "Zip Code", 
    "fully_vaccinated_percent": "% Fully Vax"
}

tooltip_dict = {
    "county": "County: ",
    "zipcode": "Zip Code",
    "at_least_one_dose_percent": "% 1+ dose", 
    "fully_vaccinated_percent": "% fully vax"
}


colormap_cutoff = [
    0, 0.2, 0.4, 0.6, 0.8, 1
]

colorscale = branca.colormap.StepColormap(
                colors=["#CDEAF8", "#97BFD6", "#5F84A9", 
                        "#315174", "#17375E",
                       ], 
                index=colormap_cutoff,
                vmin=0, vmax=1,
)

fig = make_maps.make_choropleth_map(vax_by_zipcode.drop(columns = "date"), 
                                    plot_col = "fully_vaccinated_percent", 
                                    popup_dict = popup_dict, 
                                    tooltip_dict = tooltip_dict, 
                                    colorscale = colorscale, 
                                    fig_width = 570, fig_height = 700, 
                                    zoom=6, centroid = [36.2, -119.1])


<a id=vax_map></a>
#### Full Vaccination Rates by Zip Code

In [ ]:
display(Markdown("% Fully Vaccinated by Zip Code"))
display(colorscale)
fig

In [ ]:
zipcode_dropdown = ipywidgets.Dropdown(description="Zip Code", 
                                      options=sorted(vax_by_zipcode.zipcode.unique()),
                                       value=90012)
    
def make_map_show_table(x):
    plot_col = "fully_vaccinated_percent"

    popup_dict = {
        "county": "County",
        "zipcode": "Zip Code", 
        "fully_vaccinated_percent": "% Fully Vax"
    }

    tooltip_dict = {
        "county": "County: ",
        "zipcode": "Zip Code",
        "at_least_one_dose_percent": "% 1+ dose", 
        "fully_vaccinated_percent": "% fully vax"
    }

    colormap_cutoff = [
        0, 0.2, 0.4, 0.6, 0.8, 1
    ]

    colorscale = branca.colormap.StepColormap(
                    colors=["#CDEAF8", "#97BFD6", "#5F84A9", 
                            "#315174", "#17375E",
                           ], 
                    index=colormap_cutoff,
                    vmin=0, vmax=1,
    )

    fig_width = 300
    fig_height = 300
    zoom = 12
    df = vax_by_zipcode.copy()
    
    subset_df = (df[df.zipcode==x]
                 .assign(
                     # When calculating centroids, use EPSG:2229, but when mapping, put it back into EPSG:4326
                     # https://gis.stackexchange.com/questions/372564/userwarning-when-trying-to-get-centroid-from-a-polygon-geopandas
                     lon = df.geometry.centroid.x,
                     lat = df.geometry.centroid.y,
                     county_partial_vax_avg = neighborhood_utils.calculate_county_avg(df, 
                                                                                     group_by="county",
                                                                                     output_col = "at_least_one_dose_percent"), 
                     county_full_vax_avg = neighborhood_utils.calculate_county_avg(df, 
                                                                                  group_by = "county",
                                                                                  output_col = "fully_vaccinated_percent"),
                     at_least_one_dose_percent = round(df.apply(lambda x: x.at_least_one_dose_percent * 100, axis=1), 0),
                     fully_vaccinated_percent = round(df.apply(lambda x: x.fully_vaccinated_percent * 100, axis=1), 0),
                 ).drop(columns = "date")
                )
    
    display_cols = ["county", "zipcode", "population", 
                   "% 1+ dose", "% fully vax", 
                   "county_partial_vax_avg", "county_full_vax_avg",
                   ]
    
    table = (subset_df.rename(columns = {
                     "at_least_one_dose_percent": "% 1+ dose", 
                     "fully_vaccinated_percent": "% fully vax",})
             [display_cols].style.format({
                '% 1+ dose': "{:.0f}%", 
                '% fully vax': "{:.0f}%", 
                'date': '{:%-m-%d-%y}', 
                'population': '{:,.0f}',
                'county_partial_vax_avg': '{:.0f}%',
                'county_full_vax_avg': '{:.0f}%',
            }).set_table_attributes("style='display:inline'")
        .hide_index()
    )
    
    display_html(table)
    
    center = [subset_df.lat, subset_df.lon]
    
    fig = make_maps.make_choropleth_map(subset_df, 
            plot_col, popup_dict, tooltip_dict, 
            colorscale, fig_width, fig_height, zoom, center)
    
    display(fig)

ipywidgets.interact(make_map_show_table, x=zipcode_dropdown)


[Return to top](#counties_by_region)